In [120]:
import random
class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE

class ReviewContainer:
    
    def __init__(self, reviews):
        self.reviews = reviews
        
    def getText(self):
        return [x.text for x in self.reviews]
    
    def getSentiment(self):
        return [x.sentiment for x in self.reviews]
    
    def evenlyDistribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
  
        random.shuffle(self.reviews)

## Load Data

In [121]:
import json

file_name = './books_small_1000.json'

reviews =[]
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        
        reviews.append(Review(review['reviewText'], review['overall']))
        
reviews[5].text

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

## Prep Data

In [122]:
len(reviews)

10000

In [123]:
from sklearn.model_selection import train_test_split

training, test =train_test_split(reviews, test_size = 0.33, random_state = 42)

train_container = ReviewContainer(training)
test_container = ReviewContainer(test)

len(cont.reviews)

872

In [133]:
train_container.evenlyDistribute()
test_container.evenlyDistribute()

train_x = train_container.getText()
train_y = train_container.getSentiment()

test_x = test_container.getText()
test_y = test_container.getSentiment()


train_y.count(Sentiment.POSITIVE)
train_y.count(Sentiment.NEGATIVE)

436

### We use bag of words vectorization
See documentation for details

Basically, we create matrices based on presence of words in our sentences
We will use these matrices to create our model

In [134]:
from sklearn.feature_extraction.text import CountVectorizer
 

CountVec = CountVectorizer()
#transform
train_x_vectors = CountVec.fit_transform(train_x)
test_x_vectors = CountVec.transform(test_x)

## Classification


#### A Set of data into classes
### Linear SVM


In [135]:
from sklearn import svm

clf_svm =  svm.SVC(kernel = 'linear')

clf_svm.fit(train_x_vectors, train_y)

test_x[0]
test_x_vectors[0]

clf_svm.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

### Decision Tree

In [136]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)
clf_dec.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

### Naive Bayes

In [137]:
from sklearn.naive_bayes import GaussianNB

clf_gnb = GaussianNB()
clf_gnb.fit(train_x_vectors.toarray(), train_y)
clf_gnb.predict(test_x_vectors[0].toarray())


array(['NEGATIVE'], dtype='<U8')

### Logistic Regression

In [138]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_y)

clf_log.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

## Evaluation

In [139]:
# Mean Accuracy
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
print(clf_gnb.score(test_x_vectors.toarray(), test_y))
print(clf_log.score(test_x_vectors, test_y))



0.7980769230769231
0.6298076923076923
0.6346153846153846
0.8149038461538461


In [140]:
# F1 score

from sklearn.metrics import f1_score
print(f1_score(test_y, clf_svm.predict(test_x_vectors), average = None, labels = [Sentiment.POSITIVE, Sentiment.NEUTRAL, 
                                                                            Sentiment.NEGATIVE]))
print(f1_score(test_y, clf_dec.predict(test_x_vectors), average = None, labels = [Sentiment.POSITIVE, Sentiment.NEUTRAL, 
                                                                            Sentiment.NEGATIVE]))
print(f1_score(test_y, clf_gnb.predict(test_x_vectors.toarray()), average = None, labels = [Sentiment.POSITIVE, Sentiment.NEUTRAL, 
                                                                            Sentiment.NEGATIVE]))
print(f1_score(test_y, clf_log.predict(test_x_vectors), average = None, labels = [Sentiment.POSITIVE, Sentiment.NEUTRAL, 
                                                                            Sentiment.NEGATIVE]))

[0.8028169  0.         0.79310345]
[0.63507109 0.         0.62439024]
[0.59574468 0.         0.66666667]
[0.82051282 0.         0.808933  ]


F:\Installation folder\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
F:\Installation folder\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
F:\Installation folder\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
F

In [132]:
train_y[0:5]

['NEGATIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE']